In [1]:
pip install transformers==4.47.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [4]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, load_dataset
import pandas as pd


In [5]:
from datasets import load_dataset, Dataset

# Load dataset directly from JSON file
# Make sure your file is uploaded to Colab (or in the same folder in Jupyter)
dataset = load_dataset("json", data_files="/content/sample_data/dataset_pastho_all2.json")

# Preview dataset
dataset


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'difficulty'],
        num_rows: 2620
    })
})

In [6]:
# 80% train, 10% validation, 10% test
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_valid = dataset['train'].train_test_split(test_size=0.1, seed=42)

train_dataset = train_valid['train']
valid_dataset = train_valid['test']
test_dataset  = dataset['test']

print("Train size:", len(train_dataset))
print("Validation size:", len(valid_dataset))
print("Test size:", len(test_dataset))


Train size: 1886
Validation size: 210
Test size: 524


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
model_name = "google/mt5-base"
tokenizer = MT5Tokenizer.from_pretrained(model_name, legacy=False)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
def tokenize(example):
    inputs = tokenizer(example["context"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(example["question"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = [(l if l != tokenizer.pad_token_id else -100) for l in targets["input_ids"]]
    return inputs

train_dataset = dataset["train"].map(tokenize, batched=True, remove_columns=dataset["train"].column_names)
# valid_dataset = dataset["validation"].map(tokenize, batched=True, remove_columns=dataset["validation"].column_names)
test_dataset  = dataset["test"].map(tokenize, batched=True, remove_columns=dataset["test"].column_names)


Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

Map:   0%|          | 0/524 [00:00<?, ? examples/s]

In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [13]:
import torch  # Make sure this is imported

from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5_pashto_qg_base",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_steps=100,
    save_steps=500,
    fp16=True if torch.cuda.is_available() else False,
    save_total_limit=2,
    predict_with_generate=True,
    remove_unused_columns=False  # keep all columns
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer
)


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1553508834.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [14]:
trainer.train()


Step,Training Loss
100,0.000000
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


TrainOutput(global_step=1572, training_loss=0.0, metrics={'train_runtime': 925.2775, 'train_samples_per_second': 6.796, 'train_steps_per_second': 1.699, 'total_flos': 7539602076205056.0, 'train_loss': 0.0, 'epoch': 3.0})

In [15]:
trainer.save_model("./mt5_pashto_qg_base")
tokenizer.save_pretrained("./mt5_pashto_qg_base")


('./mt5_pashto_qg_base/tokenizer_config.json',
 './mt5_pashto_qg_base/special_tokens_map.json',
 './mt5_pashto_qg_base/spiece.model',
 './mt5_pashto_qg_base/added_tokens.json',
 './mt5_pashto_qg_base/tokenizer.json')

In [16]:
def generate_question(context):
    input_text = f"generate question: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output_ids = model.generate(
        **inputs,
        max_length=64,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

# Example usage
print(generate_question("د پښتو ژبې د ویونکو شمیر ډېر دی."))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<extra_id_0>:: <extra_id_1>: <extra_id_2>: <extra_id_3>: <extra_id_4> e: <extra_id_16>. <extra_id_24>


In [29]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

model_path = "/content/mt5_pashto_qg_base"  # path where your model is saved
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [30]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_question(context):
    input_text = f"generate question: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)

    output_ids = model.generate(
    **inputs,
    max_length=64,          # max tokens for question
    num_beams=4,            # beam search to improve quality
    early_stopping=True,
    no_repeat_ngram_size=2, # avoid repeating words
    temperature=0.7,        # lower = more focused output
    top_k=50,               # consider top 50 candidates
    top_p=0.9               # nucleus sampling
)


    return tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)


In [31]:
context = "په نقشو کې رنګونه د ځمکې د مختلفو بڼو ښودلو لپاره کارول کېږي."
print("Generated Question:", generate_question(context))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated Question: ▁<extra_id_0> په نقشو رنګ
